![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import re

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

Хочется переименовать колонки в удобный для работы вид

In [ ]:
data.columns = ['restaurant_id','city','cuisine_style','ranking','price_range','number_of_reviews','reviews','url_ta','id_ta','sample','rating']

# Extra Data

Подгрузим данные о городах, странах, их столицах и числах населения

In [ ]:
DATA_DIR_EXTRA = '/kaggle/input/world-cities-datasets/'
df_cities_all = pd.read_csv(DATA_DIR_EXTRA+'worldcities.csv')

In [ ]:
df_cities = df_cities_all[((df_cities_all['city_ascii'].isin(data['city'].drop_duplicates()))
                          &(df_cities_all['country']!='United States')
                          &(df_cities_all['country']!='Canada')
                          &(df_cities_all['country']!='Venezuela'))
                          |(df_cities_all['city_ascii']=='Porto')
                         ]
df_cities['city_ascii'] = df_cities['city_ascii'].apply(lambda x: 'Oporto' if x == 'Porto' else x)
df_cities['capital'] = df_cities['capital'].apply(lambda x: 1 if x == 'primary' else 0)

In [ ]:
df_cities.drop(['city', 'lat', 'lng', 'iso2', 'iso3', 'admin_name', 'id'], axis = 1, inplace = True)

In [ ]:
df_cities

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

Проверим, какие данные содержат пропуски

In [ ]:
data.isna().sum()

Столбцов с пропусками не много, но в основном строк прилично, удалять их не очень хорошая затея, попробуем заполнить.

1. Number of Reviews

In [ ]:
data['number_of_reviews'].describe()

Пропуски в кол-ве отзывов могут означать отсутствие отзывов

In [ ]:
data['number_of_reviews'].fillna(0, inplace=True)

2. Price range

In [ ]:
data['price_range'].describe()

Количество отсутствующих значений для этого признака велико, пока отметим пометкой 'Other'

In [ ]:
data['price_range'].fillna('Other', inplace = True)

3. Cuisine Style

In [ ]:
data['cuisine_style'].describe()

Поступим аналогично

In [ ]:
data['cuisine_style'].fillna('Other', inplace = True)

4. Reviews

In [ ]:
data['reviews'].describe()

Заполним пропуски значением [[], []] как самым часто встречающимся. В то же время оно самое нейтральное и может говорить об отсутствии отзывов. 

In [ ]:
data['reviews'].fillna(data['reviews'].mode()[0], inplace = True)

Пропуски обработали

In [ ]:
data.info()

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

1. City (город). 31 уникальное значение, преобразовываем в dummy-переменную

In [ ]:
data = pd.get_dummies(data, columns=[ 'city',], dummy_na=True)

2. Price Range (ценовой сегмент)

In [ ]:
data['price_range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
price_dict = {'Other':1,'$':2,'$$ - $$$':3,'$$$$':4,}
data.price_range = data.price_range.replace(to_replace=price_dict)
display(data.price_range.value_counts())

3. Cuisine Style (кухня). 

In [ ]:
# Очищаем данные от квадратных скобок
data['cuisine_style'] = data['cuisine_style'].apply(lambda x: re.sub(r'[\[\]\'\s+]', '', str(x)))
# Преобразовываем значения колонки cuisine_style в список
data['cuisine_style_list'] = data['cuisine_style'].str.split(',')

In [ ]:
data_cuisine = data.explode('cuisine_style_list')

In [ ]:
# С использованием модуля MultiLabelBinarizer получаем dummy-переменные из cuisine_style_list
mlb = MultiLabelBinarizer()
data = data.join(pd.DataFrame(mlb.fit_transform(data.pop('cuisine_style_list')), index=data.index, columns=mlb.classes_))

In [ ]:
# Формируем датафрейм кухня - количество ресторанов
cuisine_df = pd.DataFrame(data_cuisine.cuisine_style_list.value_counts()).reset_index()

In [ ]:
# Оставим только те виды кухонь, которые формируют 95% выборки
perc_95 = np.percentile(data_cuisine.cuisine_style_list.value_counts(), 95)
cuisine_for_del = cuisine_df[(cuisine_df['cuisine_style_list']<perc_95) & (cuisine_df['index']=='Other')]['index']
for cuisine in cuisine_for_del:
    data.drop(cuisine, axis=1, inplace=True) 

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

Functions

In [ ]:
# функция получения dummy-переменных из cuisine_style
def get_cuisine_dummies(data):
    data_cuisine = data.explode('cuisine_style_list')
    # С использованием модуля MultiLabelBinarizer получаем dummy-переменные из cuisine_style_list
    mlb = MultiLabelBinarizer()
    data = data.join(pd.DataFrame(mlb.fit_transform(data.pop('cuisine_style_list')), index=data.index, columns=mlb.classes_))
    # Формируем датафрейм кухня - количество ресторанов
    cuisine_df = pd.DataFrame(data_cuisine.cuisine_style_list.value_counts()).reset_index()
    # Оставим только те виды кухонь, которые формируют 95% выборки
    perc_95 = np.percentile(data_cuisine.cuisine_style_list.value_counts(), 95)
    cuisine_for_del = cuisine_df[(cuisine_df['cuisine_style_list']<perc_95) & (cuisine_df['index']=='Other')]['index']
    for cuisine in cuisine_for_del:
        data.drop(cuisine, axis=1, inplace=True)
        
    return data

In [ ]:
# получить общее кол-во кухонь
def get_cuisine_cnt(data):
    data_cuisine = data.explode('cuisine_style_list')
    cuisine_df = pd.DataFrame(data_cuisine.cuisine_style_list.value_counts()).reset_index()
    
    return len(cuisine_df)

In [ ]:
# получить агрегаты по атрибуту ranking в рамках city
def get_max_rank_in_city(data):
    df_max_rank_in_city = pd.DataFrame(data.groupby(data['city']).ranking.agg(['mean', 'max']).reset_index())
    df_max_rank_in_city.columns = ['city', 'avg_rank', 'max_rank']
    
    return df_max_rank_in_city

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    df_output.columns = ['restaurant_id','city','cuisine_style','ranking','price_range','number_of_reviews','reviews','url_ta','id_ta','sample','rating']
    df_output = df_output.merge(df_cities, left_on='city', right_on='city_ascii')
    
    df_max_rank_city = get_max_rank_in_city(df_output)
    df_output = df_output.merge(df_max_rank_city, how = 'left', on='city')
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['restaurant_id','id_ta',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    df_output['number_of_reviews'].fillna(0, inplace=True)
    df_output['price_range'].fillna('Other', inplace = True)
    df_output['cuisine_style'].fillna('Other', inplace = True)
    df_output['reviews'].fillna(df_output['reviews'].mode()[0], inplace = True)
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'country',], dummy_na=True)
    # Обработка price_range
    price_dict = {'Other':1,'$':2,'$$ - $$$':3,'$$$$':4,}
    df_output.price_range = df_output.price_range.replace(to_replace=price_dict)
#     df_output = pd.get_dummies(df_output, columns=[ 'price_range',], dummy_na=True)
    
    # Обработка population
    df_output['population'] = df_output['population']/1000
    
    # Обработка cuisine_style
    # Очищаем данные от квадратных скобок
    df_output['cuisine_style'] = df_output['cuisine_style'].apply(lambda x: re.sub(r'[\[\]\'\s+]', '', str(x)))
    # Преобразовываем значения колонки cuisine_style в список
    df_output['cuisine_style_list'] = df_output['cuisine_style'].str.split(',')

    # ################### 4. Feature Engineering ####################################################
    df_output['cuisine_style_cnt'] = df_output['cuisine_style_list'].apply(lambda x: len(x))
    cuisine_cnt = get_cuisine_cnt(df_output)
    # какая часть всех стилей кухонь представлена в ресторане
    df_output['cuisine_style_cnt_related'] = df_output['cuisine_style_cnt'].apply(lambda x: x/cuisine_cnt)
    
    df_output['ranking_relates_population'] = df_output['ranking']/df_output['population']
    
    df_output['rank_relates_max_rank'] = df_output['ranking']/df_output['max_rank']
    df_output['rank_relates_avg_rank'] = df_output['ranking']/df_output['avg_rank']
    
#     df_output = get_cuisine_dummies(df_output)
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values            # наш таргет
X = train_data.drop(['rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
